## Money creation example 2: Money creation and destruction through lending
We demonstrate how money is created through bank lending and destroyed through repayment, and the effects of interest payments.

We start by importing required utilities. Technical notes: the code assumes that the notebook is saved in the money creation examples folder, if you wish to save it in a different folder modify the path below accordingly (if you wish to execute or modify the code herein, it can be viewed without that). Furthermore, the cells containing booking statements should only be executed once, because the booking statements will be recorded on each execution, which may result in assertion errors, e.g. when more cash is deducted than the agent possesses. If you wish to execute the booking statements again, then all cells starting from the declaration of the agents should be executed again, in order to reset the system and repeat the sequence of booking statements.

In [1]:
import os
import sys

base_path = os.path.realpath(os.getcwd()+"/../..")
sys.path.append(base_path)

from abcFinance import Ledger, Account, AccountSide

## Declaration of agents
We start by defining their agents and the accounts on their balance sheets:

In [2]:
bank = Ledger(residual_account_name="Equity")
private_agent = Ledger(residual_account_name="Equity")

bank.make_asset_accounts(['Cash','Loans','Reserves'])
bank.make_liability_accounts(['Deposits'])
bank.make_flow_accounts(['Interest income'])

private_agent.make_asset_accounts(['Cash','Deposits'])
private_agent.make_liability_accounts(['Loans'])
private_agent.make_flow_accounts(['Dividend income','Interest expenses'])

We further define a function that computes the money stocks according to our defined taxonomy:

In [3]:
def print_money_stocks():
    # Bank money: bank liabilities that are money
    bank_money = bank.get_balance('Deposits')[1]
        
    print("Total (Bank) Money:",bank_money)

def print_balance_sheets_and_money_stocks():
    print("Bank balance sheet:")
    bank.print_balance_sheet()
    print("Private agent balance sheet:")
    private_agent.print_balance_sheet()
    print_money_stocks()

## Start of the example
We start by endowing the bank and the private agent with money (note that for the sake of this example it does not matter whether the bank is a commercial or central bank).

In [4]:
bank.book(debit=[('Reserves',100)],credit=[('Equity',10),('Deposits',90)])
private_agent.book(debit=[('Deposits',90)],credit=[('Equity',90)])
bank.book_end_of_period()
private_agent.book_end_of_period()
print_balance_sheets_and_money_stocks()

Bank balance sheet:
Asset accounts:
   Reserves : 100
Liability accounts:
   Deposits : 90
Equity:  10
Total Assets:  100
--
Private agent balance sheet:
Asset accounts:
   Deposits : 90
Liability accounts:
Equity:  90
Total Assets:  90
--
Total (Bank) Money: 90


The bank now grants a loan to the private agent, thereby increasing the stock of total money:

In [5]:
bank.book(debit=[('Loans',100)],credit=[('Deposits',100)])
private_agent.book(debit=[('Deposits',100)],credit=[('Loans',100)])
print_balance_sheets_and_money_stocks()

Bank balance sheet:
Asset accounts:
   Loans : 100
   Reserves : 100
Liability accounts:
   Deposits : 190
Equity:  10
Total Assets:  200
--
Private agent balance sheet:
Asset accounts:
   Deposits : 190
Liability accounts:
   Loans : 100
Equity:  90
Total Assets:  190
--
Total (Bank) Money: 190


The private agent now pays some interest on its loan to the bank. Note that the money stock (temporarily) decreases, since the private agent uses its deposits to pay interest, thereby contributing to the bank's profit (and in this case, since there are no other expenses, equity).

In [6]:
private_agent.book(debit=[('Interest expenses',5)],credit=[('Deposits',5)])
bank.book(debit=[('Deposits',5)],credit=[('Interest income',5)])
print("Bank P&L and change in capital:")
bank.print_profit_and_loss()
print("Private agent P&L and change in capital:")
private_agent.print_profit_and_loss()
bank.book_end_of_period()
private_agent.book_end_of_period()
print_balance_sheets_and_money_stocks()

Bank P&L and change in capital:
Flow accounts:
   Interest income : 5
Profit for period:  5
--
Private agent P&L and change in capital:
Flow accounts:
   Interest expenses : -5
Profit for period:  -5
--
Bank balance sheet:
Asset accounts:
   Loans : 100
   Reserves : 100
Liability accounts:
   Deposits : 185
Equity:  15
Total Assets:  200
--
Private agent balance sheet:
Asset accounts:
   Deposits : 185
Liability accounts:
   Loans : 100
Equity:  85
Total Assets:  185
--
Total (Bank) Money: 185


The private agent now repays the loan principal, thereby (permanently) destroying the money that was created through the loan granting:

In [7]:
private_agent.book(debit=[('Loans',100)],credit=[('Deposits',100)])
bank.book(debit=[('Deposits',100)],credit=[('Loans',100)])
print_balance_sheets_and_money_stocks()

Bank balance sheet:
Asset accounts:
   Reserves : 100
Liability accounts:
   Deposits : 85
Equity:  15
Total Assets:  100
--
Private agent balance sheet:
Asset accounts:
   Deposits : 85
Liability accounts:
Equity:  85
Total Assets:  85
--
Total (Bank) Money: 85


The bank now transfers its profit from the period to the bank owners in the form of dividends, thereby increasing the money stock again. In this example the transfer goes back to the same private agent, but in practice the borrowers and owners of the bank would typically be different sets of agents. Note that in practice the bank's profit is also moved to other agents in a variety of other (expense) forms, e.g. as payment for purchases the bank makes and as salary to its employees.

In [8]:
bank.book(debit=[('Equity',5)],credit=[('Deposits',5)],text='Dividend payout')
private_agent.book(debit=[('Deposits',5)],credit=[('Dividend income',5)])
print("Bank P&L and change in capital:")
bank.print_profit_and_loss()
print("Private agent P&L and change in capital:")
private_agent.print_profit_and_loss()
bank.book_end_of_period()
private_agent.book_end_of_period()
print_balance_sheets_and_money_stocks()

Bank P&L and change in capital:
Flow accounts:
Profit for period:  0
Profit distribution and capital actions
   Dividend payout : -5
--
Private agent P&L and change in capital:
Flow accounts:
   Dividend income : 5
Profit for period:  5
--
Bank balance sheet:
Asset accounts:
   Reserves : 100
Liability accounts:
   Deposits : 90
Equity:  10
Total Assets:  100
--
Private agent balance sheet:
Asset accounts:
   Deposits : 90
Liability accounts:
Equity:  90
Total Assets:  90
--
Total (Bank) Money: 90
